In [51]:
import requests
import datetime
from datetime import date
import calendar
import time
import json
import pandas as pd 
import numpy as np

In [69]:
response = requests.get("https://api.coinranking.com/v1/public/coin/1/history/30d")
jason = response.json()
time = datetime.datetime.now()


dta = []

for x in jason['data']['history']:
    dta.append(x)

df = pd.DataFrame(dta)

df2 = df

df2['timestamp'] = df2['timestamp'] / 1000.0

df3 = df2

print(datetime.datetime.fromtimestamp(df2['timestamp'][0]))

for index, row in df2.iterrows():
    df3['timestamp'][index] = datetime.datetime.fromtimestamp(df2['timestamp'][index])

df3['price'] = df3['price'].astype(float)

hours = []

for index, row in df3.iterrows():
    hours.append(df3['timestamp'][index].hour)

df3['hours'] = hours


# SCHEMA 2#

daily = []
average = []
variance = []
total = 0
volatility = []
done = False

schema2 = df3.copy(deep=True)

start = False

for index,row in schema2.iterrows():
       
    schema2['price'][index] = round(schema2['price'][index],2)

    daily.append(schema2['price'][index])
        
    if schema2['hours'][index] == 0:
        average.append(np.average(daily))
        variance.append(np.var(daily))
        stdev = np.std(daily)
        for x in daily:
            if (abs(x-np.average(daily))/stdev) > 2:
                volatility.append('true')
                done = True
                break
        if done == False:
            volatility.append('false')
        done = False
        daily.clear()

schema2 = schema2.loc[schema2['hours'] == 0]


schema2['dailyAverage'] = average
schema2['dailyVariance'] = variance
schema2['volatilityAlert'] = volatility
schema2.rename(columns={'timestamp':'date'},inplace=True)
    
    


# SCHEMA 1#

df3 = df3.loc[df3['hours'] == 0]

df3 = df3.reset_index(drop=True)

direction = []
day = []
change = []
high = []
low = []
highIdx = 0
lowIdx = 0

for index, row in df3.iterrows():
    
    day.append(calendar.day_name[df3['timestamp'][index].weekday()])
    df3['price'][index] = round(df3['price'][index],2)
    
    if index == 0:
        direction.append('na')
        change.append('na')
        high.append('na')
        low.append('na')
        #highIdx.append('na')
        #lowIdx.append('na')
    else:
        change.append(df3['price'][index]-df3['price'][index-1])
        
        if change[index] == 0:
            direction.append('same')
        elif change[index] < 0:
            direction.append('down')
        else:
            direction.append('up')
        
        if df3['price'][index] > df3['price'][highIdx]:
            high[highIdx] = 'false'
            highIdx = index
            high.append('true')
        else:
            high.append('false')
        
        if df3['price'][index] < df3['price'][lowIdx]:
            low[lowIdx] = 'false'
            lowIdx = index
            low.append('true')
        else:
            low.append('false')

df3['direction'] = direction
df3['change'] = change
df3['dayOfWeek'] = day
df3['highSinceStart']= high
df3['lowSinceStart'] = low
df3.rename(columns={'timestamp':'date'},inplace=True)




2021-04-11 18:00:00


/Users/varunjain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varunjain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varunjain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [66]:
df3.head()

,price,date,hours,direction,change,dayOfWeek,highSinceStart,lowSinceStart
0,60968.92,2021-04-12 00:00:00,0,na,na,Monday,false,false
1,60933.23,2021-04-13 00:00:00,0,down,-35.69,Tuesday,false,false
2,64186.61,2021-04-14 00:00:00,0,up,3253.38,Wednesday,true,false
3,63831.43,2021-04-15 00:00:00,0,down,-355.18,Thursday,false,false
4,63109.69,2021-04-16 00:00:00,0,down,-721.74,Friday,false,false


In [68]:
schema2.head()


,price,date,hours,dailyAverage,dailyVariance,volatilityAlert
6,60968.92,2021-04-12 00:00:00,0,60542.848571,43171.748555,true
30,60933.23,2021-04-13 00:00:00,0,60683.408750,101172.730036,false
54,64186.61,2021-04-14 00:00:00,0,63159.390417,865131.993796,true
78,63831.43,2021-04-15 00:00:00,0,63999.029583,731408.053979,false
102,63109.69,2021-04-16 00:00:00,0,63384.759583,105481.417229,false


In [71]:
df3.to_csv(r'./files')
schema2.to_csv(r'./files')